## 3. Setting Environment(Colab)

In [ ]:
# Colab 환경 설정  # 라이브러리 설치 
# requirements : https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt

!pip install mxnet
!pip install gluonnlp==0.8.0 
!pip install pandas tqdm   
!pip install sentencepiece
!pip install transformers
!pip install torch

In [11]:
# 깃허브에서 KoBERT 파일 로드 ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-08prq54w/kobert-tokenizer_29eefb57bc0145dcae97b70fc5fb158e
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-08prq54w/kobert-tokenizer_29eefb57bc0145dcae97b70fc5fb158e
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [12]:
# import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [13]:
# Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# 경로설정
PATH = ""

In [ ]:
# pre-trained 모델 가져오기
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [14]:
# class BERTSentenceTransform

class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

## Data Preparation and Training

## 데이터셋 전처리

In [ ]:
# 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 학습할 데이터셋 불러오기
import pandas as pd
df = pd.read_csv(PATH + 'dataset.csv')

In [ ]:
# 감정별 넘버링
### 행복, 놀람, 분노, 공포, 혐오, 슬픔, 중립
df.loc[(df['Emotion'] == "행복"), 'Emotion'] = 0  #행복 => 0
df.loc[(df['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
df.loc[(df['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df.loc[(df['Emotion'] == "공포"), 'Emotion'] = 3  #공포 => 3
df.loc[(df['Emotion'] == "혐오"), 'Emotion'] = 4  #혐오 => 4
df.loc[(df['Emotion'] == "슬픔"), 'Emotion'] = 5  #슬픔 => 5
df.loc[(df['Emotion'] == "중립"), 'Emotion'] = 6  #중립 => 6

In [21]:
# 문장과 감정만 담은 새로운 데이터 프레임 생성
data_list = []
for q, label in zip(df['Sentence'], df['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

##3. 데이터 분리 (train&test)

In [ ]:
# train set, test set 분리
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

print(len(dataset_train)) 
print(len(dataset_test)) 

In [24]:
# class BERTDataset
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
# 세팅 파라미터
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  3e-5


위에서 정의된 BERTSPTokenizer를 바탕으로  train, test 전처리

In [26]:
# 데이터 로더
data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False) 
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False) 

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 5. KoBERT 학습모델

In [27]:
#device - GPU 설정
device = torch.device("cuda:0")

In [28]:
# class BERTClassifier
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)



In [ ]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

In [ ]:
#모델 체크 포인터
model = torch.load(PATH + 'checkpoint.pt')

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

## 6. 모델 학습시키기

In [30]:
## 모델 save_pointer
def save_checkpoint(epoch, model, optimizer, filename):
    state = {
        'Epoch': epoch,
        'State_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, filename)

In [31]:
# 모델 학습
import time

s = time.time()

eval_acc = 0.0

val_loss_li = []
val_acc_li = []
train_loss_li = []
train_acc_li = []



for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length

        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    train_loss_li.append(loss.data.cpu().numpy())
    train_acc_li.append(train_acc / (batch_id+1))

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation loss {} validation acc {}".format(e+1, loss.data.cpu().numpy() ,test_acc / (batch_id+1)))

    # 검증 데이터에 대한 학습 진행 상황을 저장
    val_loss_li.append(loss.data.cpu().numpy())
    val_acc_li.append(test_acc / (batch_id+1))

    if test_acc > eval_acc:
        eval_acc = test_acc
        torch.save(model,PATH + 'checkpoint.pt')  # 전체 모델 저장
        torch.save(model.state_dict(), PATH + 'model_state.pt')  # 모델 객체의 state_dict 저장
        torch.save({
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()
        },PATH + 'all.tar') # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능


e = time.time()
print(e-s, "초")

<ipython-input-31-c82a87869642>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1974 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.110177755355835 train acc 0.140625
epoch 1 batch id 201 loss 1.7211813926696777 train acc 0.21548507462686567
epoch 1 batch id 401 loss 1.0674560070037842 train acc 0.3439058603491272
epoch 1 batch id 601 loss 1.1103515625 train acc 0.42473481697171384
epoch 1 batch id 801 loss 0.8018521070480347 train acc 0.4886665106117353
epoch 1 batch id 1001 loss 0.9317035675048828 train acc 0.5326704545454546
epoch 1 batch id 1201 loss 0.8820563554763794 train acc 0.5672356369691923
epoch 1 batch id 1401 loss 0.8631917834281921 train acc 0.591920949321913
epoch 1 batch id 1601 loss 0.9419474005699158 train acc 0.6115708931917552
epoch 1 batch id 1801 loss 0.8565548062324524 train acc 0.6281059133814547
epoch 1 train acc 0.6395126534656417


<ipython-input-31-c82a87869642>:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 6 validation loss 0.047752704471349716 validation acc 0.7946579188355505


  0%|          | 0/1974 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.425462931394577 train acc 0.875
epoch 7 batch id 201 loss 0.2885848879814148 train acc 0.9095149253731343
epoch 7 batch id 401 loss 0.2799135744571686 train acc 0.9112764962593516
epoch 7 batch id 601 loss 0.2717629373073578 train acc 0.9108777038269551
epoch 7 batch id 801 loss 0.31295692920684814 train acc 0.9133309925093633
epoch 7 batch id 1001 loss 0.1596633642911911 train acc 0.9137268981018981
epoch 7 batch id 1201 loss 0.19827386736869812 train acc 0.9153049542048293
epoch 7 batch id 1401 loss 0.270393431186676 train acc 0.9155402391149179
epoch 7 batch id 1601 loss 0.3888218402862549 train acc 0.9152970799500312
epoch 7 batch id 1801 loss 0.2943870425224304 train acc 0.9156978761799001
epoch 7 train acc 0.9154714412360689


  0%|          | 0/494 [00:00<?, ?it/s]

epoch 7 validation loss 0.033001407980918884 validation acc 0.7957980889724311
9368.60168170929 초


In [ ]:
model = torch.load(PATH + 'checkpoint.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

## 7. 결과물 테스트

In [ ]:
# 예측 함수
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    all_test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)


    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(all_test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

    out = model(token_ids, valid_length, segment_ids)

# 행복, 놀람, 분노, 공포, 혐오, 슬픔, 중립
    test_eval=[]
    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()

        if np.argmax(logits) == 0:
            test_eval.append("행복이")
        elif np.argmax(logits) == 1:
            test_eval.append("놀람이")
        elif np.argmax(logits) == 2:
            test_eval.append("분노가")
        elif np.argmax(logits) == 3:
            test_eval.append("공포가")
        elif np.argmax(logits) == 4:
            test_eval.append("혐오가")
        elif np.argmax(logits) == 5:
            test_eval.append("슬픔이")
        elif np.argmax(logits) == 6:
            test_eval.append("중립이")

    print(test_eval[0] + " 느껴집니다.")

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")